# arena

In [1]:
%run -n main.py
records = pd.read_parquet('data/arena/train-00000-of-00001-cced8514c7ed782a.parquet').itertuples()
instruction_items = {
    _['instruction']: _
    for _ in parse_arena(records)
    if _['lang'] == 'English'
}
arena_items = list(instruction_items.values())
random.sample(arena_items, 3)

[{'id': '8afb4907c8a44f109ac887ea3992f68c',
  'lang': 'English',
  'instruction': 'Write every possible combination of the numbers 1, 2 and 3 (that is 1,2,3  3,1,2 - etc)  - except the ones where the second number is larger than the last number. The numbers can be repeated, that is 3,1,1 is a valid combination.'},
 {'id': 'bc5bc3ff98a8454b843cb001787a11ee',
  'lang': 'English',
  'instruction': 'A riddle. It starts digging, replaces a hundred shovels. What is it?'},
 {'id': 'cbf55ba157ed4287a9c916afbb570437',
  'lang': 'English',
  'instruction': 'What is consciousness? Is thought dependent on the physical presence of neurons, or are those things independent of each other?'}]

# translate

In [2]:
%run -n main.py
dotenv = read_json('.env.json')
openai.api_key = dotenv['OPENAI_TOKEN']

In [ ]:
%run -n main.py
translate_items = [
    {
        'id': _['id']
        'instruction': _['instruction'],
        'answer': None
    }
    for _ in random.sample(arena_items, 1500)
]

In [ ]:
%run -n main.py
items = [_ for _ in translate_items if not _['answer']][:250]
queue = iter(tqdm(items))
workers = [openai_translate_worker(queue) for _ in range(10)]
await asyncio.gather(*workers);

# manually check

In [ ]:
arena_ru_items = [
    {
        'id': _['id'],
        'instruction': _['answer'].splitlines()
    }
    for _ in translate_items
    if _['answer']
]    
write_json('arena_ru.json', arena_ru_items)

In [14]:
items = read_json('arena_ru.json')
for item in items:
    item['instruction'] = '\n'.join(item['instruction'])
write_jsonl('arena_ru.jsonl', items)